In [103]:
import requests as rq
import pandas as pd
import json
import re
from os import path
import numpy as np
import os
import time


headers='''GET /list_detail_rate.htm?itemId=547501969350&sellerId=2963804430&currentPage=3 HTTP/1.1
Host: rate.tmall.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:63.0) Gecko/20100101 Firefox/63.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2
Accept-Encoding: gzip, deflate, br
Referer: http://47.98.102.15:8000/user/dwhang/notebooks/project/Advancy/CommentCloud/Untitled.ipynb
Connection: keep-alive
Cookie: isg=BMTEvbdqRLJhXvb4ucBMiOWllkJ2deghlAxfyN5lAA9sCWTTBu8r12PrTWF0CiCf; cna=DxZfEOFgfSACAWVQIak4S5J3; um=A502B1276E6D5FEFA4FA8106E94F56A71182424073D91A416F8F7DD6D52553BBBB53A858D2A6095ACD43AD3E795C914CE2613C9E5B7EAEDA7234B2176B533EA1; enc=%2BRieSqjS7Kse1Dg0FtXGyfKCidraf7EyLOOgLH0NnkkP%2BgS25knRvEKJxrluxZ%2FCmlVHD4ei5jKJB21GEPnEdA%3D%3D; lid=rhmily0627; otherx=e%3D1%26p%3D*%26s%3D0%26c%3D0%26f%3D0%26g%3D0%26t%3D0; x=__ll%3D-1%26_ato%3D0; hng=CN%7Czh-CN%7CCNY%7C156; _m_h5_tk=e4d5466d15fcc6d6ea9cc27bf72619cb_1543853986491; _m_h5_tk_enc=51049a4890510b49f00095f9c3da42a8; t=d7a679ff82efaa327a852200cdfaa45e; uc3=vt3=F8dByR1Vf9HTjZkhcS4%3D&id2=Vvz0JWcxsM4C&nk2=EU90aKraUlbnig%3D%3D&lg2=VT5L2FSpMGV7TQ%3D%3D; tracknick=rhmily0627; lgc=rhmily0627; _tb_token_=ee3be0e13a6e7; cookie2=19612e8b60bc591f8fd3e14839ac57e4; x5sec=7b22726174656d616e616765723b32223a226166376231613130636532313666373635373636633632613534313764623661434c66336c4f4146454f4b7435714b327339446a44673d3d227d
Upgrade-Insecure-Requests: 1
Cache-Control: max-age=0'''

# 去除参数头尾的空格并按换行符分割
headers = headers.strip().split('\n')
 
# 使用字典生成式将参数切片重组，并去掉空格，处理带协议头中的://
headers = {x.split(':')[0].strip(): ("".join(x.split(':')[1:])).strip().replace('//', "://") for x in headers}
cookies=headers['Cookie']
cookies={x.split('=')[0].strip():("".join(x.split('=')[1:])).strip() for x in cookies.split(';')}
        
def get_TM_pinglun(url):
    #url='http://rate.tmall.com/list_detail_rate.htm?itemId= 41464129793&sellerId= 1652490016&currentPage= 1'
    print(url)
    myweb=rq.get(url,headers=headers,cookies=cookies)
    myjson=re.findall('"rateList".*',myweb.text)
    txt=re.findall('"rateList":(.*)',myweb.text)[0]
    reptxt=re.findall(',"searchinfo":"","from":"search","paginator":{"lastPage":\d*,"page":\d*,"items":\d*},"tags":\[\]}}\)',txt)[0]
    tt=txt.replace(reptxt,'').replace('[{','').replace('}]','').split('},{')
    Data=[]
    for i,a in enumerate(tt):
        temp='{"A":{'+a+'}}'
        temp=pd.read_json(temp).T
        if i==0:
            Data=temp
        else:
            Data=Data.append(temp,ignore_index=True)
    return(Data)



def get_TM_item_pinglun(itemId,sellerId,pages,file='word1.txt'):
    #itemId=re.findall(re.compile("itemId=\d*"),url)[0].replace('itemId=','')
    #sellerId=re.findall(re.compile("sellerId=\d*"),url)[0].replace('sellerId=','')
    #itemId=re.findall(re.compile('id=\d*'),url)[0].replace('id=','')
    #sellerId=re.findall(re.compile('user_id=\d*'),url)[0].replace('user_id=','')
    rawurl='http://rate.tmall.com/list_detail_rate.htm?itemId=%s&sellerId=%s&currentPage=%s'
    Data=[]
    unpage=list()
    for num in range(1,pages+1):
        time.sleep(1)
        url=rawurl%(itemId,sellerId,num)
        try:
            temp=get_TM_pinglun(url)
            #print(url)
            if len(Data)==0:
                Data=temp
            else:
                Data=Data.append(temp,ignore_index=True)
        except:
            unpage.append(num)
    #Data=Data[['rateContent','rateDate','goldUser','gmtCreateTime','tradeEndTime','useful']].drop_duplicates()
    Data['rateContent']=Data['rateContent'].apply(lambda x: x.replace('&hellip',''))
    os.chdir('/home/dwhang/project/Advancy/CommentCloud/Data')
    Data.to_csv(file, index=False, header=True,encoding='utf-8')
    print(unpage)
    return

def get_JD_pinglun(productId,page):
    s=rq.session()
    url = 'https://club.jd.com/comment/productPageComments.action'
    data = { 'productId':'1994763191',
            'score':0,
            'sortType':5,
            'pageSize':100,
            'isShadowSku':0,
            'page':0
    }
    data['productId']=productId
    data['page']=page
    r = s.get(url, params = data)
    txt=r.json()['comments']
    Data=list()
    for a in txt:
        Data.append(a)
    return(Data)
    
def get_JD_item_pinglun(url,pages,file='word1.txt'):
    itemId=re.findall('\d*[0-9]',url)[0]
    Data=list()
    unpage=list()
    for num in range(0,pages):
        time.sleep(1)
        try:
            temp=get_JD_pinglun(itemId,num)
            Data=Data+temp
        except:
            unpage.append(num)
    Data=pd.DataFrame(Data)
    os.chdir('/home/dwhang/project/Advancy/CommentCloud/Data')
    Data.to_csv(file,header=True,encoding='utf-8')
    print(unpage)
    return


In [ ]:
url='https://rate.tmall.com/list_detail_rate.htm?itemId=541238084688&spuId=851483444&sellerId=2928278102&order=3&currentPage=2&append=0&content=1&tagId=&posi=&picture=&groupId=&ua=098%23E1hv69vBvoWvUvCkvvvvvjiPRLq9gjEvPFc9sjrCPmPOgjDRPFzw0jrbnLFpQjtbRphvCvvvvvmCvpvZzH1V3ZLNznswpWrfMAfGva1j7IVrvpvEvv1JyNPsvVA4dphvmpvUwgYhBvCw%2BFyCvvpvvvvvCQhvCYsw7DdNMMhtvpvhvvvvvvhCvvOvCvvvphmEvpvV2vvC9jX2uphvmvvv9bLPi%2B9wKphv8PMMvjAMpv2Fvvm2phCv2WUvvUnvphvpgvvv96CvpCCvvvm2ohCvhUvEvpCWvvyCvvaAQWoHyoTTwhVXQjjxVE9kBjjxQWp1VB1iQWoEzmTTwhVHzjjxVE93pzjxQWp1VC1iQWoXQmTTwhVEycjxVE9khzjxQWp1epGCvvpvvPMMiQhvCvvv9U8jvpvhvvpvv86Cvvyv2WAhA%2BIvVhRrvpvEvC2KDwd8vU9K&needFold=0&_ksTS=1552889239010_3791&callback=jsonp3792'
itemId=re.findall('itemId=\d*',url)[0].replace('itemId=','')
file='Tmall'+itemId+'.xlsx'
rawurl='http://rate.tmall.com/list_detail_rate.htm?itemId=%s&sellerId=%s&currentPage=%s'
Data=[]
pages=99
unpage=list()
for num in range(1,pages+1):
    time.sleep(5)
    url=rawurl%(itemId,sellerId,num)
    try:
        temp=get_TM_pinglun(url)
        #print(url)
        if len(Data)==0:
            Data=temp
        else:
            Data=Data.append(temp,ignore_index=True)
    except:
        unpage.append(num)
#Data=Data[['rateContent','rateDate','goldUser','gmtCreateTime','tradeEndTime','useful']].drop_duplicates()
#Data['rateContent']=Data['rateContent'].apply(lambda x: x.replace('&hellip',''))
#os.chdir('/home/dwhang/project/Advancy/CommentCloud/Data')
Data.to_excel(file)

http://rate.tmall.com/list_detail_rate.htm?itemId=541238084688&sellerId=2928278102&currentPage=1


In [ ]:
url='https://rate.tmall.com/list_detail_rate.htm?itemId=579781006119&spuId=1090402183&sellerId=2928278102&order=3&currentPage=2&append=0&content=1&tagId=&posi=&picture=&groupId=&ua=098%23E1hvnpvPvBvvUpCkvvvvvjiPRLq9gjEvRsqy0j3mPmPWQj1bP2Spgj1UnLSwtjlUPghCvCB4cFPBga147DumJJawiDSk75MNh46CvCvwvnO9ajpv9CVnrsw6Zf5BdphvmpvpFBquT9C%2BP2yCvvpvvvvvCQhvCli4zYMwPvytvpvhvvvvvvhCvvOvCvvvphmtvpvIMMGvNbYvvRWvvUnvphvWQ9vv96CvpC29vvm2phCvhhvvvUnUphvp9UyCvvOWvvVvayRivpvUvvmvrW%2Fe41OEvpCWCCYTvvaKYR2WlEy7nDeDyO2v5fh3Zi7v1R9t%2BFBCAfevD46fjomxfa3lDBhS2fmDYWoXeznv%2BneYiLUpwhKn3w0xhCDTmEcBlwyzhvGCvvpvvPMMiQhvCvvv9U8jvpvhvvpvv2yCvvpvvvvv&needFold=0&_ksTS=1552889283370_3778&callback=jsonp3779'
itemId=re.findall('itemId=\d*',url)[0].replace('itemId=','')
file='Tmall'+itemId+'.xlsx'
rawurl='http://rate.tmall.com/list_detail_rate.htm?itemId=%s&sellerId=%s&currentPage=%s'
Data=[]
pages=100
unpage=list()
for num in range(1,pages+1):
    time.sleep(5)
    url=rawurl%(itemId,sellerId,num)
    try:
        temp=get_TM_pinglun(url)
        #print(url)
        if len(Data)==0:
            Data=temp
        else:
            Data=Data.append(temp,ignore_index=True)
    except:
        unpage.append(num)
#Data=Data[['rateContent','rateDate','goldUser','gmtCreateTime','tradeEndTime','useful']].drop_duplicates()
#Data['rateContent']=Data['rateContent'].apply(lambda x: x.replace('&hellip',''))
#os.chdir('/home/dwhang/project/Advancy/CommentCloud/Data')
Data.to_excel(file)

In [102]:
url='https://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&spuId=711141539&sellerId=2928278102&order=3&currentPage=2&append=0&content=1&tagId=&posi=&picture=&groupId=&ua=098%23E1hvxvvRvPQvUvCkvvvvvjiPRLq9gjECP2zp6j1VPmPwgjEVP2SpzjtbPLsWQjE89phvHn1wSIvVzYswz2%2FE7M%2F8Mvzw9HuCRphvCvvvvvvPvpvhvv2MMQyCvhQpPRhvCsalYWmQrEt1B%2F2p%2BnezrmphQRAn3feAOHj9TWexRdIAcUmxfwpXd56bS47B9C97Ecqh6jc6VCOiHRLysRp7RAYVyO2vqbVQKphv8PMMvjAMpv2Fvvm2phCv2WUvvUnvphvpgvvv96CvpCCvvvm2ohCvhUmEvpvV2vvC9jX2uphvmvvv9bLPioZ5vphvC9vhvvCvp2yCvvpvvvvviQhvCvvv9U8jvpvhvvpvvvwCvvNwzHi4zQ9qdphvmpvpQIySbQvgRFyCvvpvvvvv&needFold=0&_ksTS=1552889308530_3814&callback=jsonp3815'
itemId=re.findall('itemId=\d*',url)[0].replace('itemId=','')
file='Tmall'+itemId+'.xlsx'
rawurl='http://rate.tmall.com/list_detail_rate.htm?itemId=%s&sellerId=%s&currentPage=%s'
Data=[]
pages=100
unpage=list()
for num in range(1,pages+1):
    time.sleep(5)
    url=rawurl%(itemId,sellerId,num)
    try:
        temp=get_TM_pinglun(url)
        #print(url)
        if len(Data)==0:
            Data=temp
        else:
            Data=Data.append(temp,ignore_index=True)
    except:
        unpage.append(num)
#Data=Data[['rateContent','rateDate','goldUser','gmtCreateTime','tradeEndTime','useful']].drop_duplicates()
#Data['rateContent']=Data['rateContent'].apply(lambda x: x.replace('&hellip',''))
#os.chdir('/home/dwhang/project/Advancy/CommentCloud/Data')
Data.to_excel(file)

http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=1
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=2
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=3
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=4
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=5
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=6
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=7
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=8
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=9
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=10
http://rate.tmall.com/list_de

http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=85
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=86
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=87
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=88
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=89
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=90
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=91
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=92
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=93
http://rate.tmall.com/list_detail_rate.htm?itemId=558188550944&sellerId=2928278102&currentPage=94
http://rate.tmall.co

In [99]:
file='Tmall'+itemId+'.xlsx'
Data.to_excel(file)

In [ ]:
url='https://rate.tmall.com/list_detail_rate.htm?itemId=568446002811&spuId=1044967259&sellerId=2928278102&order=3&currentPage=2&append=0&content=1&tagId=&posi=&picture=&groupId=&ua=098%23E1hviQvWvRyvUvCkvvvvvjiPRLq9gjEVRszWljD2PmPZQj3vR2LZ0jDnPLFhQjDURphvChCvvvm5vpvhphvhH2yCvvBvpvvv2QhvCvvvMM%2FtvpvIS6vvgMMMpb3vpCQmvvCmYhCvjvUvvhBGphvwv9vvBHBvpC1mvvChX8yCvv3vpvoD1s430UyCvvX2pQ9hVtIEvpCWpiBOv8W78BpDN%2BLWdiYtLjCVTRmfjjX4nDeDyBvOJ193Zi7veCrTmmxBlwyzhmyZEcqht8gJEcttEbk1%2BFw5ZA9c1WvwezEUaycWgvhCvvXvovvvvvmjvpvhphhvv86CvvDvBR0yWQCvLU4CvpvZzHAT3NPNznswjSEfMgdGPYAv7Ih%3D&needFold=0&_ksTS=1552889476208_3799&callback=jsonp3800'
itemId=re.findall('itemId=\d*',url)[0].replace('itemId=','')
file='Tmall'+itemid+'.xlsx'
rawurl='http://rate.tmall.com/list_detail_rate.htm?itemId=%s&sellerId=%s&currentPage=%s'
Data=[]
pages=100
unpage=list()
for num in range(1,pages+1):
    time.sleep(5)
    url=rawurl%(itemId,sellerId,num)
    try:
        temp=get_TM_pinglun(url)
        #print(url)
        if len(Data)==0:
            Data=temp
        else:
            Data=Data.append(temp,ignore_index=True)
    except:
        unpage.append(num)
#Data=Data[['rateContent','rateDate','goldUser','gmtCreateTime','tradeEndTime','useful']].drop_duplicates()
#Data['rateContent']=Data['rateContent'].apply(lambda x: x.replace('&hellip',''))
#os.chdir('/home/dwhang/project/Advancy/CommentCloud/Data')
Data.to_excel(file)

In [14]:
Data.to_excel('559534190631.xlsx')

In [15]:
url_n='https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50095527&pageIndex=1&score=0&_19031108'
url_good='https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50095527&pageIndex=1&score=1&_19031108'
url_bad='https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50095527&pageIndex=1&score=-1&_19031108'
pages=[1052,7,4]

In [72]:
from bs4 import BeautifulSoup
def get_comment(url,tp='bad'):
    s=rq.session()
    try:
        web=s.get(url)
        web.encoding='gbk'
        soup=BeautifulSoup(web.text,'lxml')
        txt=[x for x in soup.select('td p') if x not in soup.select('.reply_txt')]
        data=pd.DataFrame({'comment':[x.get_text() for x in txt[::2]],'date':[x.get_text() for x in txt[1::2]],'type':tp})
        return(data)
    except Exception as e :
        print(e)
    

In [90]:
goods=50096855
Data=pd.DataFrame()
unpage=[]
base_url='https://www.111.com.cn/interfaces/review/list/html.action?goodsId=%s&pageIndex=%s&score=%s&_19031108'
for score,tp,pages in zip([-1,0,1],['bad','normal','good'],[5,7,580]):
    for page in range(1,pages+1):
        url=base_url%(goods,page,score)
        data=get_comment(url,tp=tp)
        print(url)
        time.sleep(2)
        if isinstance(data,pd.DataFrame)==True:
            Data=Data.append(data,ignore_index=True)
        else:
            unpage.append(page)
Data.to_excel('111_50096855.xlsx')

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=1&score=-1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=2&score=-1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=3&score=-1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=4&score=-1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=5&score=-1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=1&score=0&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=2&score=0&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=3&score=0&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=4&score=0&_19031108
https://www.111.com.cn/interfaces/review/list/html

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=67&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=68&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=69&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=70&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=71&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=72&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=73&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=74&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=75&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=144&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=145&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=146&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=147&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=148&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=149&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=150&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=151&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=152&score=1&_19031108
https://www.111.com.cn/interfaces/rev

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=221&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=222&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=223&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=224&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=225&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=226&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=227&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=228&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=229&score=1&_19031108
https://www.111.com.cn/interfaces/rev

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=298&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=299&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=300&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=301&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=302&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=303&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=304&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=305&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=306&score=1&_19031108
https://www.111.com.cn/interfaces/rev

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=375&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=376&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=377&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=378&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=379&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=380&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=381&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=382&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=383&score=1&_19031108
https://www.111.com.cn/interfaces/rev

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=452&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=453&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=454&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=455&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=456&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=457&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=458&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=459&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=460&score=1&_19031108
https://www.111.com.cn/interfaces/rev

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=529&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=530&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=531&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=532&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=533&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=534&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=535&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=536&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50096855&pageIndex=537&score=1&_19031108
https://www.111.com.cn/interfaces/rev

In [91]:
goods=50179202
Data1=pd.DataFrame()
unpage=[]
base_url='https://www.111.com.cn/interfaces/review/list/html.action?goodsId=%s&pageIndex=%s&score=%s&_19031108'
for score,tp,pages in zip([-1,0,1],['bad','normal','good'],[0,0,29]):
    for page in range(1,pages+1):
        url=base_url%(goods,page,score)
        data=get_comment(url,tp=tp)
        print(url)
        time.sleep(3)
        if isinstance(data,pd.DataFrame)==True:
            Data1=Data1.append(data,ignore_index=True)
        else:
            unpage.append(page)
Data1.to_excel('111_50179202.xlsx')

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=1&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=2&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=3&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=4&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=5&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=6&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=7&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=8&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50179202&pageIndex=9&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.acti

In [92]:
goods=50008806
Data2=pd.DataFrame()
unpage=[]
base_url='https://www.111.com.cn/interfaces/review/list/html.action?goodsId=%s&pageIndex=%s&score=%s&_19031108'
for score,tp,pages in zip([-1,0,1],['bad','normal','good'],[3,2,47]):
    for page in range(1,pages+1):
        url=base_url%(goods,page,score)
        data=get_comment(url,tp=tp)
        print(url)
        time.sleep(3)
        if isinstance(data,pd.DataFrame)==True:
            Data2=Data2.append(data,ignore_index=True)
        else:
            unpage.append(page)
Data2.to_excel('111_50008806.xlsx')

https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=1&score=-1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=2&score=-1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=3&score=-1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=1&score=0&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=2&score=0&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=1&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=2&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=3&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.action?goodsId=50008806&pageIndex=4&score=1&_19031108
https://www.111.com.cn/interfaces/review/list/html.a